# Tanzania water pumps project : Machine Learning

In [55]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score, log_loss
from scipy.stats import kendalltau
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

In [56]:
clean_df = pd.read_csv('../Data/Clean_DF.csv', index_col='id')

In [57]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 23 columns):
amount_tsh               59400 non-null float64
funder                   59400 non-null object
gps_height               59400 non-null int64
installer                59400 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
basin                    59400 non-null object
region                   59400 non-null object
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           59400 non-null bool
permit                   59400 non-null bool
construction_year        59400 non-null float64
extraction_type_class    59400 non-null object
management               59400 non-null object
payment                  59400 non-null object
quality_group            59400 non-null obje

## Categorize columns

In [58]:
X = pd.DataFrame()
# These categories are not ordered
col_to_cat=['funder','installer','basin','region','district_code','lga','ward',
            'extraction_type_class','management','source_type','waterpoint_type_group']
for col in col_to_cat:
    X[col] = clean_df[col].astype('category').cat.codes

# These categories are ordered
payment_type = pd.CategoricalDtype(categories=['other','never pay','pay when scheme fails',
                                               'pay annually','pay monthly','pay per bucket'],
                                   ordered=True)
X['payment'] = clean_df['payment'].astype(payment_type).cat.codes

quality_group_type = pd.CategoricalDtype(categories=['unknown','fluoride','salty','colored',
                                                     'milky','good'], ordered=True)
X['quality_group'] = clean_df['quality_group'].astype(quality_group_type).cat.codes

quantity_type = pd.CategoricalDtype(categories=['unknown','dry','insufficient','seasonal',
                                                'enough'], ordered=True)
X['quantity'] = clean_df['quantity'].astype(quantity_type).cat.codes

In [59]:
X = pd.concat([X, clean_df[['public_meeting','permit']]], axis='columns')

## Scale numeric columns

In [60]:
col_numeric = ['amount_tsh','gps_height','longitude','latitude','population','construction_year']

In [61]:
# Scale columns using MinMax scaler to have only positive values
s = MinMaxScaler()
scaled_array = s.fit_transform(clean_df[col_numeric].astype(float))
scaled_df = pd.DataFrame(scaled_array, index=clean_df.index, columns=col_numeric)

XS = pd.concat([X, scaled_df], axis='columns')

In [62]:
# X is not scaled
X = pd.concat([X, clean_df[col_numeric]], axis='columns')

In [63]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 22 columns):
funder                   59400 non-null int8
installer                59400 non-null int8
basin                    59400 non-null int8
region                   59400 non-null int8
district_code            59400 non-null int8
lga                      59400 non-null int8
ward                     59400 non-null int8
extraction_type_class    59400 non-null int8
management               59400 non-null int8
source_type              59400 non-null int8
waterpoint_type_group    59400 non-null int8
payment                  59400 non-null int8
quality_group            59400 non-null int8
quantity                 59400 non-null int8
public_meeting           59400 non-null bool
permit                   59400 non-null bool
amount_tsh               59400 non-null float64
gps_height               59400 non-null int64
longitude                59400 non-null float64
latitude                 

In [64]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
funder,59400.0,50.852559,26.907972,0.000000,27.000000,61.000000,66.000000,100.000000
installer,59400.0,44.419680,24.918253,0.000000,27.000000,33.000000,67.000000,99.000000
basin,59400.0,4.077172,2.473082,0.000000,2.000000,4.000000,6.000000,8.000000
region,59400.0,9.835606,5.936892,0.000000,4.000000,10.000000,15.000000,20.000000
district_code,59400.0,3.867424,2.992263,0.000000,2.000000,3.000000,5.000000,19.000000
lga,59400.0,60.039882,35.244499,0.000000,30.000000,60.000000,91.000000,124.000000
ward,59400.0,89.999798,23.952799,0.000000,100.000000,100.000000,100.000000,100.000000
extraction_type_class,59400.0,1.264663,1.626676,0.000000,0.000000,1.000000,2.000000,6.000000
management,59400.0,6.329630,1.917857,0.000000,6.000000,6.000000,6.000000,10.000000
source_type,59400.0,3.970556,2.191798,0.000000,3.000000,5.000000,6.000000,6.000000


In [65]:
XS.describe().T

,count,mean,std,min,25%,50%,75%,max
funder,59400.0,50.852559,26.907972,0.0,27.000000,61.000000,66.000000,100.0
installer,59400.0,44.419680,24.918253,0.0,27.000000,33.000000,67.000000,99.0
basin,59400.0,4.077172,2.473082,0.0,2.000000,4.000000,6.000000,8.0
region,59400.0,9.835606,5.936892,0.0,4.000000,10.000000,15.000000,20.0
district_code,59400.0,3.867424,2.992263,0.0,2.000000,3.000000,5.000000,19.0
lga,59400.0,60.039882,35.244499,0.0,30.000000,60.000000,91.000000,124.0
ward,59400.0,89.999798,23.952799,0.0,100.000000,100.000000,100.000000,100.0
extraction_type_class,59400.0,1.264663,1.626676,0.0,0.000000,1.000000,2.000000,6.0
management,59400.0,6.329630,1.917857,0.0,6.000000,6.000000,6.000000,10.0
source_type,59400.0,3.970556,2.191798,0.0,3.000000,5.000000,6.000000,6.0


In [66]:
# Label
y = pd.Series()
labels = ['non functional','functional needs repair', 'functional']
status_group_type = pd.CategoricalDtype(categories=labels, ordered=True)
y = clean_df['status_group'].astype(status_group_type).cat.codes

## Separate training from hold out data

Split training and hold out data with stratify because the labels are not balanced.

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [68]:
XS_train, XS_test, ys_train, ys_test = train_test_split(XS, y, test_size=0.2, stratify=y)

## Calculate metrics for perfect model

In [69]:
def scores(y_test,y_pred,y_pred_proba=None):
    print('Confusion matrix:')
    print(confusion_matrix(y_test, y_pred))
    print('\nF1 score: ', f1_score(y_test, y_pred, average='weighted'))
    if y_pred_proba is not None:
        print('\nlog_loss: ', log_loss(y_test, y_pred_proba))
    print('\n', kendalltau(y_test, y_pred))
    return

In [70]:
print('Scores for Perfect Model:\n')
scores(y_test,y_test)

Scores for Perfect Model:

Confusion matrix:
[[4565    0    0]
 [   0  863    0]
 [   0    0 6452]]

F1 score:  1.0

 KendalltauResult(correlation=1.0, pvalue=0.0)


Kendall Tau is a measure of the correspondence between 2 rankings. It's a good measure for this problem because the labels are ordered.

## Calculate metrics for simplest model

The most simple model would be to give the label a random value according to the proportions in the sample.

In [71]:
labels_count = y_test.value_counts().sort_index()
y_pred_s = np.ones_like(y_test, dtype=int)
y_pred_s[:labels_count[0]] = 0
y_pred_s[-labels_count[2]:] = 2
np.random.shuffle(y_pred_s)

The probabilities for each label are the proportions of each label in the sample.

In [72]:
y_pred_proba_s = np.empty((len(y_test),3),dtype=float)
for i in range (0,3):
    y_pred_proba_s[:,i] = labels_count[i]/len(y_test)

In [73]:
print('Scores for Simplest Model:\n')
scores(y_test, y_pred_s, y_pred_proba_s)

Scores for Simplest Model:

Confusion matrix:
[[1708  333 2524]
 [ 334   58  471]
 [2523  472 3457]]

F1 score:  0.4396464646464647

log_loss:  0.8895473153415451

 KendalltauResult(correlation=-0.01596640706638022, pvalue=0.06557633629012284)


## Calculate metrics for different models

### Linear Logistic Regression

In [74]:
lr = OneVsRestClassifier(LogisticRegression(random_state=14, multi_class='ovr',
                                            class_weight='balanced', solver='liblinear'))

In [75]:
lr.fit(XS_train, ys_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=None, penalty='l2', random_state=14,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None)

In [76]:
y_pred_lr = lr.predict(XS_test)
y_pred_proba_lr = lr.predict_proba(XS_test)
print('Scores for logistic regression:\n')
scores(ys_test, y_pred_lr, y_pred_proba_lr)

Scores for logistic regression:

Confusion matrix:
[[2979  591  995]
 [ 224  277  362]
 [1378 1142 3932]]

F1 score:  0.6280878216993391

log_loss:  0.9389346419568432

 KendalltauResult(correlation=0.3990733338814309, pvalue=0.0)


### SVM

In [77]:
svm = OneVsRestClassifier(SVC(random_state=14, class_weight='balanced', 
                              decision_function_shape='ovr', gamma='scale'))

In [78]:
# use scaled XS for SVM
svm.fit(XS_train, ys_train)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=14, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=None)

In [79]:
y_pred_svm = svm.predict(XS_test)
print('Scores for SVM:\n')
scores(ys_test, y_pred_svm)

Scores for SVM:

Confusion matrix:
[[3033  604  928]
 [ 145  509  209]
 [1093 1084 4275]]

F1 score:  0.6793574645879152

 KendalltauResult(correlation=0.47387505081048015, pvalue=0.0)


### Random Forest

In [80]:
rf = OneVsRestClassifier(RandomForestClassifier(random_state=14, n_estimators=10))

In [81]:
rf.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=14, verbose=0, warm_start=False),
          n_jobs=None)

In [82]:
y_pred_rf = rf.predict(X_test)
y_pred_proba_rf = rf.predict_proba(X_test)

In [83]:
print('Scores for Random Forest:\n')
scores(y_test, y_pred_rf, y_pred_proba_rf)

Scores for Random Forest:

Confusion matrix:
[[3494   82  989]
 [ 129  275  459]
 [ 551  185 5716]]

F1 score:  0.7916885376282825

log_loss:  1.8730356603260692

 KendalltauResult(correlation=0.6497368919366576, pvalue=0.0)


### K Neighbors

In [84]:
kn = OneVsRestClassifier(KNeighborsClassifier())

In [85]:
# Use scaled XS
kn.fit(XS_train, ys_train)

OneVsRestClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
          n_jobs=None)

In [86]:
y_pred_kn = kn.predict(XS_test)
y_pred_proba_kn = kn.predict_proba(XS_test)

In [87]:
print('Scores for K Neighbors\n')
scores(ys_test, y_pred_kn, y_pred_proba_kn)

Scores for K Neighbors

Confusion matrix:
[[3176   90 1299]
 [ 125  271  467]
 [ 756  186 5510]]

F1 score:  0.7469809089442876

log_loss:  2.703396730475956

 KendalltauResult(correlation=0.5553090601109151, pvalue=0.0)


### Naive Bayes

In [88]:
nb = OneVsRestClassifier(MultinomialNB())

In [89]:
# Use scaled XS for Naive Bayes
nb.fit(XS_train,ys_train)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          n_jobs=None)

In [90]:
y_pred_nb = nb.predict(XS_test)
y_pred_proba_nb = nb.predict_proba(XS_test)

In [91]:
print('Scores for Naive Bayes\n')
scores(ys_test,y_pred_nb, y_pred_proba_nb)

Scores for Naive Bayes

Confusion matrix:
[[2778  283 1504]
 [ 284  108  471]
 [2430  416 3606]]

F1 score:  0.5471867176939406

log_loss:  1.6980747296735936

 KendalltauResult(correlation=0.2094623795997861, pvalue=1.320371631122597e-128)


### AdaBoost

In [92]:
ab = OneVsRestClassifier(AdaBoostClassifier(random_state=14))

In [93]:
ab.fit(X_train, y_train)

OneVsRestClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=14),
          n_jobs=None)

In [94]:
y_pred_ab = ab.predict(X_test)
y_pred_proba_ab = ab.predict_proba(X_test)

In [95]:
print('Scores for Ada Boost:\n')
scores(y_test, y_pred_ab, y_pred_proba_ab)

Scores for Ada Boost:

Confusion matrix:
[[2813   17 1735]
 [ 158   45  660]
 [ 624   33 5795]]

F1 score:  0.7016497705535681

log_loss:  1.0790205292006343

 KendalltauResult(correlation=0.5129807923719177, pvalue=0.0)


## Optimize the best model: Random Forest

### Display the model attributes

In [96]:
features_df = pd.DataFrame(index=list(X.columns))
i=0
features_df["average"] = 0
for estimator in rf.estimators_:
    i+=1
    features_df["Random Forest " + str(i)] = estimator.feature_importances_
    features_df["average"] += estimator.feature_importances_    
features_df["average"] /= i
print('Features importance\n')
features_df

Features importance



,average,Random Forest 1,Random Forest 2,Random Forest 3
funder,0.042505,0.038512,0.046738,0.042264
installer,0.032801,0.032075,0.034779,0.031548
basin,0.016130,0.016639,0.016010,0.015742
region,0.019179,0.018564,0.017005,0.021969
district_code,0.022212,0.021684,0.022378,0.022575
lga,0.032430,0.034127,0.030449,0.032716
ward,0.014638,0.011832,0.018194,0.013886
extraction_type_class,0.040134,0.059185,0.021547,0.039671
management,0.019607,0.021181,0.018135,0.019505
source_type,0.021304,0.023900,0.018368,0.021643


### Try to reduce the number of features to simplify the model.

public_meeting and permit have less importance

In [97]:
XR_train = X_train.drop(columns=['public_meeting','permit'])
XR_test = X_test.drop(columns=['public_meeting','permit'])

In [98]:
rf.fit(XR_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=14, verbose=0, warm_start=False),
          n_jobs=None)

In [99]:
y_pred_rf = rf.predict(XR_test)
y_pred_proba_rf = rf.predict_proba(XR_test)

In [100]:
print('Scores for Random Forest with less features:\n')
scores(y_test, y_pred_rf, y_pred_proba_rf)

Scores for Random Forest with less features:

Confusion matrix:
[[3515   77  973]
 [ 134  278  451]
 [ 584  189 5679]]

F1 score:  0.7908188403975204

log_loss:  1.7929986014093628

 KendalltauResult(correlation=0.6469134788655483, pvalue=0.0)


F1 score and Kendall Tau correlation have the same values. 

** Conclusion: I can get rid of the features: public_meeting and permit.**

### Use K Fold to optimize parameters

I keep XR_test, y_test as hold out, and I optimize the parameters using XR_train, y_train.

In [101]:
def fit_predict_score(XR_train, y_train, model, p, scores_df):
    """Make K folds, fit model, predict labels and returns f1 and kendall tau scores"""
    n_splits = 5
    kf = StratifiedKFold(n_splits=n_splits)
    for train_idx, test_idx in kf.split(XR_train, y_train):
        XKF_train, XKF_test = XR_train.iloc[train_idx], XR_train.iloc[test_idx]
        ykf_train, ykf_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
        model.fit(XKF_train, ykf_train)
        ykf_pred = model.predict(XKF_test)
        f1 = f1_score(ykf_test, ykf_pred, average='weighted')
        kt, pval = kendalltau(ykf_test, ykf_pred)
        scores_df.loc[p,'f1 score'] += f1
        scores_df.loc[p,'kendall tau'] += kt
    scores_df.loc[p,'f1 score'] /= n_splits
    scores_df.loc[p,'kendall tau'] /= n_splits
    return scores_df

First, I will optimize n_estimators

In [102]:
n_estimators_grid = [10,50,100,150,200,300]

In [103]:
scores_df = pd.DataFrame(0, index=n_estimators_grid, columns=['f1 score','kendall tau'])
for p in n_estimators_grid:
    rf = OneVsRestClassifier(RandomForestClassifier(n_estimators=p))
    scores_df = fit_predict_score(XR_train, y_train, rf, p, scores_df)
scores_df

,f1 score,kendall tau
10,0.789091,0.643934
50,0.793284,0.652081
100,0.794525,0.654158
150,0.794825,0.653897
200,0.794657,0.653808
300,0.795858,0.656789


There is a good improvement from 10 to 50, so I choose 50 for n_estimators.

Then, I will optimize max_depth

In [104]:
max_depth_grid = [10,50,100,150,200]

In [105]:
scores_df = pd.DataFrame(0, index= max_depth_grid, columns=['f1 score','kendall tau'])
for p in max_depth_grid:
    rf = OneVsRestClassifier(RandomForestClassifier(n_estimators=50, max_depth=p))
    scores_df = fit_predict_score(XR_train, y_train, rf, p, scores_df)
scores_df

,f1 score,kendall tau
10,0.737661,0.585219
50,0.794157,0.653380
100,0.793939,0.652324
150,0.794509,0.653910
200,0.794030,0.653356


I choose to use max_depth=50.

### Test the optimized model on the held out test data

In [106]:
rf = OneVsRestClassifier(RandomForestClassifier(n_estimators=50, max_depth=50))
rf.fit(XR_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          n_jobs=None)

In [107]:
y_pred_rf = rf.predict(XR_test)
y_pred_proba_rf = rf.predict_proba(XR_test)

In [108]:
print('Scores for Optimized Random Forest:\n')
scores(y_test, y_pred_rf, y_pred_proba_rf)

Scores for Optimized Random Forest:

Confusion matrix:
[[3566   76  923]
 [ 136  289  438]
 [ 589  179 5684]]

F1 score:  0.7967275094113107

log_loss:  0.8811162075756189

 KendalltauResult(correlation=0.657044266632395, pvalue=0.0)


These are the best results I got so far!